In [ ]:
import os
import random
import cv2
import matplotlib.pyplot as plt

In [ ]:
common_dir = '/kaggle/input/brain-mri-images-for-brain-tumor-detection/brain_tumor_dataset/'

In [ ]:
path = os.path.join(common_dir, 'no/')
im_loc = random.choice(os.listdir(path))
im = cv2.imread(os.path.join(path, im_loc))
print('Shape of Image:',im.shape)

In [ ]:
# displaying healthy brain image:
path_0 = os.path.join(common_dir, 'no/')
plt.figure(figsize=(15,15))
print('-'*20, 'Healthy Brain Images','-'*20)
for i in range(1,9):
    plt.subplot(2,4,i)
    img_loc = random.choice(os.listdir(path_0))
    img = cv2.imread(os.path.join(path_0, img_loc))
    plt.imshow(img)
    plt.xticks([]), plt.yticks([])

In [ ]:
# displaying tumor brain image:
path_1 = os.path.join(common_dir, 'yes/')
plt.figure(figsize=(15,15))
print('-'*20, 'Tumor Brain Images','-'*20)
for i in range(1,9):
    plt.subplot(2,4,i)
    img_loc = random.choice(os.listdir(path_1))
    img = cv2.imread(os.path.join(path_1, img_loc))
    plt.imshow(img)
    plt.xticks([]), plt.yticks([])

## Training Model

In [ ]:
# Importing Libraries for training the model:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.models import Model, Sequential
from keras.applications import MobileNetV2

In [ ]:
datagen = ImageDataGenerator(rescale=1.0/255.0,
                            brightness_range=[0.2,1],
                            zoom_range=(0.2,1),
                            vertical_flip=True,
                            horizontal_flip=True,
                            validation_split=0.1)

train_set = datagen.flow_from_directory(common_dir,
                                       target_size=(224,224),
                                       class_mode='binary',
                                       subset='training')

test_set = datagen.flow_from_directory(common_dir,
                                      target_size=(224,224),
                                      class_mode='binary',
                                      subset='validation')

In [ ]:
base_model = MobileNetV2(weights='imagenet',input_shape=(224,224,3), include_top=False,pooling='avg.')

base_model.trainable=False
model = Sequential()
model.add(base_model)
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(BatchNormalization()) # faster computational.
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_set, epochs=20)